En este notebook nos encargaremso de crear las dos nuevas tablas con ingenieria de caracteristicas una contedra la totlidd de nuevas caracteristicas qeu supondremos que mejoraran el set de entrenamiento y el otro tendra la mitad de esas caracteristicas esto es par saber si con mas es mejor cada una sera testeada despues

Claro, aquí tienes una versión mejorada, más clara, coherente y profesional del texto, junto con la fórmula de variación porcentual diaria y un ejemplo más estructurado del análisis de desfase:

---

Vamos a utilizar como plantilla base la tabla `train`, a partir de la cual crearemos dos nuevas copias denominadas `Test_50_insight` y `Test_100_insight`.

Planeo trabajar con el DataFrame `oil`, del cual extraeremos las columnas `date` y `dcoilwtico` (precio del petróleo WTI). Con esta información, calcularemos la **variación porcentual diaria del precio del petróleo** y la almacenaremos en una nueva columna llamada `oil_price`.

Esta columna será posteriormente combinada con la tabla `train`, alineando los valores por la columna `date`, para así conocer cómo varió el precio del petróleo cada día específico.

### Cálculo de la variación porcentual diaria

La fórmula para calcular la variación porcentual diaria es:

$$
\text{variación}_{t} = \frac{P_{t} - P_{t-1}}{P_{t-1}} \times 100
$$

Donde:

* $P_t$ es el precio del petróleo en el día actual
* $P_{t-1}$ es el precio del día anterior

### Justificación del desfase temporal

Inicialmente se pensó en **adelantar un día** la variable `oil_price`, ya que el precio del petróleo actual podría influir en las decisiones del mercado **al día siguiente**, no en el mismo día en que se registra el precio. Sin embargo, para tomar una decisión informada, se propone realizar un análisis de **correlación** entre la variable `oil_price` y otras variables objetivo, considerando tres escenarios distintos:

1. **Desfase -1 (precio del día anterior):**
   Se utiliza el precio del petróleo del día anterior para predecir el comportamiento del día actual.
2. **Sin desfase (precio actual):**
   Se utiliza el precio del petróleo del mismo día.
3. **Desfase +1 (precio del día siguiente):**
   Se usa el precio del día siguiente, bajo la hipótesis de que el precio de hoy refleja la reacción al comportamiento del día anterior.

### Ejemplo ilustrativo de desfases

Aquí tienes un ejemplo más claro de cómo funciona el desfase en los precios del petróleo. Supongamos que los precios diarios son los siguientes:

| Fecha      | Precio WTI |
| ---------- | ---------- |
| 17 de mayo | 1.01       |
| 18 de mayo | 1.05       |
| 19 de mayo | 1.02       |

Y que nuestra plantilla `train` contiene las mismas fechas como índice para unir:

| Fecha      | … otras columnas … |
| ---------- | ------------------ |
| 17 de mayo | …                  |
| 18 de mayo | …                  |
| 19 de mayo | …                  |

Ahora veamos los tres escenarios:

---

### 1. Desfase -1 (precio del día anterior)

Aquí asignamos al 18 de mayo el precio del 17 de mayo, y al 19 de mayo el del 18 de mayo:

| Fecha (train) | oil\_price (t-1) |
| ------------- | ---------------- |
| 17 de mayo    | —                |
| 18 de mayo    | 1.01             |
| 19 de mayo    | 1.05             |

* **Interpretación**: Para el 18 usas el precio del 17, asumiendo que el mercado reacciona con un día de retraso.

---

### 2. Sin desfase (precio del mismo día)

Asignamos a cada fecha su propio precio:

| Fecha (train) | oil\_price (t) |
| ------------- | -------------- |
| 17 de mayo    | 1.01           |
| 18 de mayo    | 1.05           |
| 19 de mayo    | 1.02           |

* **Interpretación**: El modelo ve el precio del petróleo del mismo día.

---

### 3. Desfase +1 (precio del día siguiente)

Aquí adelantamos un día: al 17 de mayo le ponemos el precio del 18, y al 18 el del 19:

| Fecha (train) | oil\_price (t+1) |
| ------------- | ---------------- |
| 17 de mayo    | 1.05             |
| 18 de mayo    | 1.02             |
| 19 de mayo    | —                |

* **Interpretación**: El modelo “ve” el precio que estará disponible mañana, ideal si crees que las señales del petróleo ya anticipan movimientos un día antes.

---

Con estos tres conjuntos de datos podrás calcular el coeficiente de correlación de cada versión de `oil_price` con tu variable objetivo y decidir cuál desfase aporta más valor predictivo.


In [4]:
# %%python
import yaml
import psycopg2
from psycopg2.extras import RealDictCursor
from pathlib import Path

# 0. (Opcional) verifica tu cwd para entender dónde estás
# import os
# print("CWD:", os.getcwd())

# 1. Construye la ruta absoluta al root del repo (un nivel arriba de notebooks)
project_root = Path.cwd().parent

# 2. Señala tu config.yaml dentro de config/
config_path = project_root / "config" / "config.yaml"

# 3. Carga el YAML
with config_path.open("r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)
db = cfg["database"]

# 4. Conecta a PostgreSQL y prueba
try:
    conn = psycopg2.connect(
        host=db["host"],
        port=db["port"],
        user=db["user"],
        password=db["password"],
        dbname=db["database"]
    )
    cur = conn.cursor(cursor_factory=RealDictCursor)
    cur.execute("SELECT version();")
    version = cur.fetchone()["version"]
    print(f"✅ Conexión exitosa. Versión de PostgreSQL: {version}")
except Exception as e:
    print(f"❌ Error al conectar: {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()


✅ Conexión exitosa. Versión de PostgreSQL: PostgreSQL 17.5 on x86_64-pc-linux-musl, compiled by gcc (Alpine 14.2.0) 14.2.0, 64-bit
